## Simulation Framework 

In [1]:
import import_ipynb
import math 

# import functions from model_evaluation.ipynb  
from model_evaluation import group_memberships
from model_evaluation import common_group
from model_evaluation import K
from model_evaluation import first_term 
from model_evaluation import interaction_terms
from model_evaluation import connection_oriented_cluster_match

importing Jupyter notebook from model_evaluation.ipynb


In [2]:
# Define input variables  
groups = [[1, 2, 3], [1], [0, 2]] 
votes = [5, 5, 5, 5]

In [3]:
# Assuming this is the correct function signature
group_memberships = group_memberships(groups, votes)
print(group_memberships)

[[2], [0, 1], [0, 2], [0]]


In [4]:
result_groups, result_aggregate_weight = first_term(groups, votes, group_memberships)
print(result_groups)
print(result_aggregate_weight)

{'group_0': {'agent_1': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}, 'agent_2': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}, 'agent_3': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}, 'group_1': {'agent_1': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}}, 'group_2': {'agent_0': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}, 'agent_2': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}}}
20.0


In [5]:
result_individual, result_aggregated = interaction_terms(groups, group_memberships, votes)
print(result_individual)
print(result_aggregated)

agent_i: 2, agent_j: 1


TypeError: 'function' object is not subscriptable

In [ ]:
model_evaluation.